# Analyze transport data
This is an example notebook for the analysis class `IV_curve` of `qkit.analysis.IV_curve.py`. This handels transport measurment data (focussed of measurements of Josephson junctions in the current bias) taken with `qkit.measure.transport.transport.py` and provides methods for
 * loading data
 * analyse voltage and current offsets
 * analyse critical currents and voltage jumps

In [ ]:
import numpy as np
from scipy import signal as sig
import matplotlib.pyplot as plt

In [ ]:
import qkit
qkit.start()

In [ ]:
from qkit.analysis.IV_curve import IV_curve as IVC
ivc = IVC()

#### read h5 file
Load transport measurement data by given `uuid`.

In [ ]:
ivc.load(uuid='XXXXXX')

#### numerical derivative $\frac{\text{d}V}{\text{d}I}$
The numerical derivative is caclulated by default using the Savitzky Golay filter, but different methods can be used, e.g. a simple numerical gradient `ivc.get_dVdI(mode=np.gradient)`

In [ ]:
ivc.get_dVdI()

### current and voltage offsets
The branch where the y-values are nearly constant are evaluated. The average of all corresponding x-values is considered to be the x-offset and the average of the extreme y-values are considered as y-offset. These are by default the critical y-values $ y_c$, but can also be set to retrapping y-values $ y_r $ if `yr=True`
 * `ivc.get_offsets()` calculates x- and y-offset of every trace,
 * `ivc.get_offset()` calculates x- and y-offset of the whole set (this differs only for 2D or 3D scans).
 
Note that reasonable initial values `offset` and `tol_offset` are sufficient.

In [ ]:
ivc.get_offsets(offset=0, tol_offset=20e-6)

In [ ]:
ivc.get_offset(offset=0, tol_offset=20e-6)

#### get $ I_c $ via threshold methode
The branch where the voltage values are nearly constant are evaluated. Their maximal values of the up- and down-sweep are considered as critical currents $ I_c $ and retrapping current $ I_r $ (if `Ir=True`), respectively.

Note that it works best, if `offset` is already determined via `get_offsets()` and that reasonable initial value `tol_offset` are sufficient.

In [ ]:
ivc.get_Ic_threshold(Ir=True)

#### get $ I_c $ via peak detection in derivative
The critical currents $ I_c $ are found by searching peaks in the numerical derivative $ \frac{\text{d}V}{\text{d}I} $. This is done using `scipy.signal.find_peaks()` by default, but can set to custom peak finding algorithms `peak_finder`.

Note that the arguments of the peak finding algorithms need to be set properly, e.g. `prominence` for `scipy.signal.find_peaks()`.

In [ ]:
I_cs, I_rs, props = ivc.get_Ic_deriv(prominence=400, Ir=True)
I_cs, I_rs

In [ ]:
print('all currents, where voltage jumps')
if ivc._scan_dim == 1:
    print(np.array(map(lambda p1D: p1D['I'], props)))
elif ivc._scan_dim == 2:
    print(np.array(map(lambda p2D: map(lambda p1D: p1D['I'], p2D), props)))
elif ivc._scan_dim == 3:
    print(np.array(map(lambda p3D: map(lambda p2D: map(lambda p1D: p1D['I'], p2D), p3D), props)))

#### get $ I_c $ via peak detection Fourier transform
The critical currents $ I_c $ are found by searching peaks in the Gaussian smoothed derivation $ \left(\text{i}f\cdot\text{e}^{-sf^2}\right) $ in the frequency domain. This is done using `scipy.signal.find_peaks()` by default, but can set to custom peak finding algorithms `peak_finder`.

Note that the smoothing factor `s` and the arguments of the peak finding algorithms need to be set properly, e.g. `prominence` for `scipy.signal.find_peaks()`.

In [ ]:
I_cs, I_rs, props = ivc.get_Ic_dft(Ir=True, prominence=1.5e-6)
I_cs, I_rs